<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/week13/13_agrawala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang-go

In [ ]:
!go version

go version go1.10.4 linux/amd64


In [14]:
%%writefile nodo.go
package main

import (
    "encoding/json"
    "fmt"
    "math/rand"
    "net"
    "os"
    "strconv"
    "time"
)

type Msg struct {
    Command     string
    Hostname    string
    List        []string
}

var friends []string
var local   string
var end     chan bool

var cont    int
var mynum   int
var nextnum int
var next    string
var imfirst bool
var ready   chan bool
var yach    chan bool

func serv() {
    fmt.Println("(", local, ")")
    ln, _ := net.Listen("tcp", local)
    defer ln.Close()
    for {
        conn, _ := ln.Accept()
        go handle(conn)
    }
}
func handle(conn net.Conn) {
    defer conn.Close()
    dec := json.NewDecoder(conn)
    var msg Msg
    if err := dec.Decode(&msg); err == nil {
        switch msg.Command {
        case "hello":
            resp := Msg{"hey", local, friends}
            enc := json.NewEncoder(conn)
            if err := enc.Encode(&resp); err == nil {
                for _, friend := range friends {
                    fmt.Println(local, "introduces", msg.Hostname,
                                "to", friend)
                    send(friend, "meet new friend", []string{msg.Hostname})
                }
            }
            friends = append(friends, msg.Hostname)
            fmt.Println(local, "updated list", friends)
        case "meet new friend":
            friends = append(friends, msg.List...)
            fmt.Println(local, "new friend", msg.List)
        
        case "start agrawala":
            mynum   = rand.Intn(100000)
            imfirst = true
            nextnum = 100001
            next    = ""
            cont    = 0
            for _, friend := range friends {
                fmt.Println(local, "sending", mynum, "to", friend)
                send(friend, "num", []string{strconv.Itoa(mynum)})
            }
            // TODO optimize this
            if len(friends) == 0 {
                go distributedCriticalSection()
                ready<-true
            }
            yach<-true
        case "num":
            <-yach
            newnum, _ := strconv.Atoi(msg.List[0])
            fmt.Println(local, cont, newnum, nextnum, next)
            if newnum < mynum {
                imfirst = false
            } else if newnum < nextnum {
                nextnum = newnum
                next = msg.Hostname
            }
            cont++
            fmt.Println(local, cont, newnum, nextnum, next, imfirst)
            if cont == len(friends) {
                if imfirst {
                    fmt.Println(local, "I'm first")
                    go distributedCriticalSection()
                }
                ready<-true
            } else {
                yach<-true
            }
        case "you can start":
            go distributedCriticalSection()
        
        case "finish":
            fmt.Println(local, "that's  all folks")
            end<-true
        }
    }
}
func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, local, list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        fmt.Println(local, "sent", msg)
        if command == "hello" {
            dec := json.NewDecoder(conn)
            var resp Msg
            if err := dec.Decode(&resp); err == nil {
                friends = append(friends, resp.List...)
                fmt.Println(local, "recibí", resp.List)
            }
        }
    }
}
func distributedCriticalSection() {
    <-ready
    fmt.Println(local, "Haciendo sección crítica!!(1)")
    fmt.Println(local, "Haciendo sección crítica!!(2)")
    fmt.Println(local, "Haciendo sección crítica!!(3)")
    fmt.Println(local, "Haciendo sección crítica!!(4)")
    fmt.Println(local, "Haciendo sección crítica!!(5)")
    if next != "" {
        send(next, "you can start", []string{})
    } else {
        for _, friend := range friends {
            send(friend, "finish", []string{})
        }
        fmt.Println(local, "I've seen things you woudln't believe")
        end<-true
    }
}
func main() {
    rand.Seed(1981)
    end     = make(chan bool)
    ready   = make(chan bool)
    yach    = make(chan bool, 1)
    local   = os.Args[1]
    go serv()
    if len(os.Args) == 3 {
        remote := os.Args[2]
        friends = append(friends, os.Args[2])
        send(remote, "hello", []string{})
    }
    go func() {
        time.Sleep(time.Second*20)
        fmt.Println(local, "executing fail safe")
        end<-true
    }()
    <-end
    fmt.Println(local, "time to die")
}

Overwriting nodo.go


In [15]:
!go build nodo.go

In [ ]:
%%writefile agrawalastarter.go
package main

import (
    "encoding/json"
    "fmt"
    "net"
    "os"
)

type Msg struct {
    Command string
    Hostname string
    List []string
}

func send(remote, command string, list []string) {
    conn, _ := net.Dial("tcp", remote)
    defer conn.Close()
    msg := Msg{command, "", list}
    enc := json.NewEncoder(conn)
    if err := enc.Encode(&msg); err == nil {
        fmt.Println("starting", remote)
    }
}

func main() {
    for _, remote := range os.Args[1:] {
        send(remote, "start agrawala", []string{})
    }
}

Writing agrawalastarter.go


In [ ]:
!go build agrawalastarter.go

In [16]:
%%script bash

./nodo localhost:8000 &
sleep 1
./nodo localhost:8001 localhost:8000 &
sleep 1

./agrawalastarter   localhost:8000 \
                    localhost:8001


( localhost:8000 )
( localhost:8001 )
localhost:8001 sent {hello localhost:8001 []}
localhost:8000 updated list [localhost:8001]
localhost:8001 recibí []
starting localhost:8000
localhost:8000 sending 41154 to localhost:8001
localhost:8000 sent {num localhost:8000 [41154]}
starting localhost:8001
localhost:8001 sending 64108 to localhost:8000
localhost:8001 sent {num localhost:8001 [64108]}
localhost:8001 0 41154 100001 
localhost:8001 1 41154 100001  false
localhost:8000 0 64108 100001 
localhost:8000 1 64108 64108 localhost:8001 true
localhost:8000 I'm first
localhost:8000 Haciendo sección crítica!!(1)
localhost:8000 Haciendo sección crítica!!(2)
localhost:8000 Haciendo sección crítica!!(3)
localhost:8000 Haciendo sección crítica!!(4)
localhost:8000 Haciendo sección crítica!!(5)
localhost:8000 sent {you can start localhost:8000 []}
localhost:8001 Haciendo sección crítica!!(1)
localhost:8001 Haciendo sección crítica!!(2)
localhost:8001 Haciendo sección crítica!!(3)
localhost:8001 Haci

In [18]:
%%script bash

./nodo localhost:8000 &
sleep 1
./nodo localhost:8001 localhost:8000 &
sleep 1
./nodo localhost:8002 localhost:8000 &
sleep 1
./nodo localhost:8003 localhost:8002 &
sleep 1
./nodo localhost:8004 localhost:8001 &
sleep 1
./nodo localhost:8005 localhost:8003 &
sleep 1

./agrawalastarter   localhost:8000 \
                    localhost:8001 \
                    localhost:8002 \
                    localhost:8003 \
                    localhost:8004 \
                    localhost:8005


( localhost:8000 )
( localhost:8001 )
localhost:8001 sent {hello localhost:8001 []}
localhost:8000 updated list [localhost:8001]
localhost:8001 recibí []
( localhost:8002 )
localhost:8002 sent {hello localhost:8002 []}
localhost:8000 introduces localhost:8002 to localhost:8001
localhost:8002 recibí [localhost:8001]
localhost:8000 sent {meet new friend localhost:8000 [localhost:8002]}
localhost:8000 updated list [localhost:8001 localhost:8002]
localhost:8001 new friend [localhost:8002]
( localhost:8003 )
localhost:8003 sent {hello localhost:8003 []}
localhost:8002 introduces localhost:8003 to localhost:8000
localhost:8003 recibí [localhost:8000 localhost:8001]
localhost:8002 sent {meet new friend localhost:8002 [localhost:8003]}
localhost:8002 introduces localhost:8003 to localhost:8001
localhost:8000 new friend [localhost:8003]
localhost:8002 sent {meet new friend localhost:8002 [localhost:8003]}
localhost:8002 updated list [localhost:8000 localhost:8001 localhost:8003]
localhost:8001 

In [ ]:
%%script bash
#kill -9 2607

ps ax

    PID TTY      STAT   TIME COMMAND
      1 ?        Ss     0:00 /bin/bash -e /datalab/run.sh
      8 ?        Sl     0:00 /tools/node/bin/node /datalab/web/app.js
     18 ?        Sl     0:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
    115 ?        Ss     0:00 tail -n +0 -F /root/.config/Google/DriveFS/Logs/drive_fs.txt
    123 ?        Ssl    0:02 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-fe4e5cb5-efe9-4d56-83f7-6d6dc32a22fe.json
   1184 ?        S      0:00 bash
   1185 ?        R      0:00 ps ax
